In [ ]:
##This code strips all the meta data from one restaurant and stores the information into a dictionary called rest_dict
from lxml import html
from bs4 import BeautifulSoup 
import os.path
import csv

#grabs the location of all restaurant html files on the local hard drive
import glob

all_restaurants_html = []

for filename in glob.iglob(r'C:\Users\Olarn\Desktop\Data Science\Springboard\Yelp_Webpages\*page_endson_0.html'):
    all_restaurants_html.append(filename)

for url in all_restaurants_html:
    #retrieve the html pages from your hard drive and then create a for loop that will go through each URL
    #and extract and compile the desired Xpaths
    tree = html.parse(url)

    #variable restaurant more business info
    with open(url,'rb') as file:
        soup = BeautifulSoup(file)

    attributes = []
    for element in soup.find_all('div',attrs={'class':'short-def-list'}):
        for subelements in element.find_all('dt',attrs={'class':'attribute-key'}):
            text = subelements.get_text()
            attributes.append(text.strip())

    values = []
    for element in soup.find_all('div',attrs={'class':'short-def-list'}):
        for subelements in element.find_all('dd'):
            text = subelements.get_text()
            values.append(text.strip())

    #create a dictionary and add the variable business information - (eg restaurant hours, bar service, ambiance)
    dictionary = dict(zip(attributes, values))
    
    #create a separate dictionary for the cuisines part of the restaurant. multiple cuisines requires a loop
    cuisine = []
    for element in soup.find('div',attrs={'class':'price-category'}).find_all('span',attrs={'class':'category-str-list'}):
        for subelements in element.find_all('a'):
            text = subelements.get_text()
            cuisine.append(text)
            
    #extract standard business information - (eg name, rating, # of reviews, location)
    name = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[3]/div[1]/div[1]/h1/text()')
    rating = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/@title')
    count = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[3]/div[1]/div[2]/div[1]/div[1]/span/text()')
    cost = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[3]/div[1]/div[2]/div[2]/span[1]/span/text()')
    price_range = tree.xpath('//*[@id="super-container"]/div/div/div[2]/div[1]/div[2]/ul/li[3]/div[2]/dl/dd/text()')
    address = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[4]/div[1]/div/div[2]/ul/li[1]/div/strong/address/text()')

    #add standard business information to existing dictionary
    dictionary['Restaurant Name'] = ''.join(name).strip()
    dictionary['Rating'] = ''.join(rating).strip()
    dictionary['Review Count'] = ''.join(count).strip()
    dictionary['Expected Cost'] = ''.join(cost).strip()
    dictionary['Price Range'] = ''.join(price_range).strip()
    dictionary['Address'] = ''.join(address).strip()
    dictionary['Cuisines'] = cuisine
    
    #create a variable for the restaurant name to be included in the file name
    restaurant_name =  dictionary['Restaurant Name']
    
    #remove characters that prevent one from creating a unique file name
    restaurant_name = restaurant_name.replace("?", "")
    
    #save the dictionary for each restaurnat into a csv file 
    save_path = r'C:\Users\Olarn\Desktop\Data Science\Springboard\Yelp_ParsedData\metadata'
    completeName = os.path.join(save_path, restaurant_name + '.csv')
    with open(completeName,'w') as handle:
        w = csv.DictWriter(handle,dictionary.keys())
        w.writeheader()
        w.writerow(dictionary)


In [ ]:
#converts all csv files saved to hard drive to a pandas data frame for testing
import csv
import glob
import pandas as pd
import chardet

#locate the meta data for restaurants extracted from the html files
path = r'C:\Users\Olarn\Desktop\Data Science\Springboard\Yelp_ParsedData\metadata'
filenames = glob.glob(os.path.join(path + '\*.csv'))

#extract the encoding information for each csv file 
enc = []
for file in filenames:
    opened_text = open(file,'rb')
    result = chardet.detect(opened_text.read())
    enc.append(result)

#read the file 
dataframe_per_file = (pd.read_csv(f, encoding = enc[filenames.index(f)]['encoding']) for f in filenames)

concatenated_df = pd.concat(dataframe_per_file, ignore_index = True)
concatenated_df = concatenated_df[['Restaurant Name', 'Rating', 'Review Count', 'Expected Cost', 'Price Range', 'Cuisines', 'Accepts Credit Cards', 'Address', 'Alcohol', 'Ambience', 'Attire', 'Bike Parking', 'Caters',  'Delivery', 'Good For', 'Good for Groups', 'Good for Kids', 'Has TV', 'Noise Level', 'Outdoor Seating','Parking', 'Take-out', 'Takes Reservations', 'Waiter Service', 'Wheelchair Accessible', 'Wi-Fi']]
print(concatenated_df.head())

In [ ]:
#maximum number of reviews that a Thai restaurant in NYC has ever received
import re
import numpy as np

#safety variable
rest_df = concatenated_df

temp_list1 = []
regex = (r'\d+')
for x in rest_df['Review Count']:
    x = str(x)
    if re.search(regex, x):
        match = re.search(regex,x)
        temp_list1.append(int(match.group(0)))
    else:
        temp_list1.append(np.nan)

rest_df['Review Count Num'] = temp_list1

In [ ]:
#cleaning the number of rating of each restaurant
import re
import numpy as np

regex = r'(\d+\.\d+)'

list = []
for x in rest_df['Rating']:
    if re.search(regex, str(x)):
        match = re.search(regex, x)
        list.append(float(match.group(0)))
    else:
        list.append(np.nan)
    
rest_df['Clean Rating'] = list

rest_df.to_csv('yelp_metadata_v0_1.csv')

In [ ]:
##This code strips all the review data from one restaurant and stores the information into a dictionary called rest_dict
from lxml import html
from bs4 import BeautifulSoup 
import os.path
import csv

#grabs the location of all restaurant html files on the local hard drive
import glob

all_restaurants_html = []

for filename in glob.iglob(r'C:\Users\Olarn\Desktop\Data Science\Springboard\Yelp_Webpages\*.html'):
    all_restaurants_html.append(filename)

for url in all_restaurants_html:
    #retrieve the html pages from your hard drive and then create a for loop that will go through each URL
    #and extract and compile the desired Xpaths
    tree = html.parse(url)

    #variable restaurant more business info
    with open(url,'rb') as file:
        soup = BeautifulSoup(file)

    attributes = []
    for element in soup.find_all('div',attrs={'class':'short-def-list'}):
        for subelements in element.find_all('dt',attrs={'class':'attribute-key'}):
            text = subelements.get_text()
            attributes.append(text.strip())

    values = []
    for element in soup.find_all('div',attrs={'class':'short-def-list'}):
        for subelements in element.find_all('dd'):
            text = subelements.get_text()
            values.append(text.strip())

    #create a dictionary and add the variable business information - (eg restaurant hours, bar service, ambiance)
    dictionary = dict(zip(attributes, values))
    
    #create a separate dictionary for the cuisines part of the restaurant. multiple cuisines requires a loop
    cuisine = []
    for element in soup.find('div',attrs={'class':'price-category'}).find_all('span',attrs={'class':'category-str-list'}):
        for subelements in element.find_all('a'):
            text = subelements.get_text()
            cuisine.append(text)
            
    #extract standard business information - (eg name, rating, # of reviews, location)
    name = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[3]/div[1]/div[1]/h1/text()')
    rating = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/@title')
    count = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[3]/div[1]/div[2]/div[1]/div[1]/span/text()')
    cost = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[3]/div[1]/div[2]/div[2]/span[1]/span/text()')
    price_range = tree.xpath('//*[@id="super-container"]/div/div/div[2]/div[1]/div[2]/ul/li[3]/div[2]/dl/dd/text()')
    address = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[4]/div[1]/div/div[2]/ul/li[1]/div/strong/address/text()')

    #add standard business information to existing dictionary
    dictionary['Restaurant Name'] = ''.join(name).strip()
    dictionary['Rating'] = ''.join(rating).strip()
    dictionary['Review Count'] = ''.join(count).strip()
    dictionary['Expected Cost'] = ''.join(cost).strip()
    dictionary['Price Range'] = ''.join(price_range).strip()
    dictionary['Address'] = ''.join(address).strip()
    dictionary['Cuisines'] = cuisine
    
    #create a variable for the restaurant name to be included in the file name
    restaurant_name =  dictionary['Restaurant Name']
    
    #remove characters that prevent one from creating a unique file name
    restaurant_name = restaurant_name.replace("?", "")
    
    #save the dictionary for each restaurnat into a csv file 
    save_path = r'C:\Users\Olarn\Desktop\Data Science\Springboard\Yelp_ParsedData\metadata'
    completeName = os.path.join(save_path, restaurant_name + '.csv')
    with open(completeName,'w') as handle:
        w = csv.DictWriter(handle,dictionary.keys())
        w.writeheader()
        w.writerow(dictionary)


In [ ]:
##This code strips all the review data from one restaurant and stores the information into a dictionary called rest_dict
from lxml import html
from bs4 import BeautifulSoup 
import os.path
import csv
import re

all_restaurants_html = []

for filename in glob.iglob(r'C:\Users\Olarn\Desktop\Data Science\Springboard\Yelp_Webpages\*page_endson_0.html'):
    all_restaurants_html.append(filename)

all_restaurants = []
for uncleaned_restaurant in all_restaurants_html[17:]:
    cleaned_restaurant_html = uncleaned_restaurant.split('_page_endson', 1)[0]
    all_restaurants.append(cleaned_restaurant_html)
    
#for loop to iterate across all restaraunts - we will iterate across a list of all resturants
for restaurant in all_restaurants:
    
    restaurant_review_num = 1
    restaurant_query = restaurant + '*'
    restaurant_review_pages = glob.iglob(restaurant_query)

    for a_review_page in restaurant_review_pages:
        #retrieve the html pages from your hard drive and then create a for loop that will go through each URL
        #and extract and compile the desired Xpaths
        tree = html.parse(a_review_page)

        #variable restaurant more business info
        with open(a_review_page,'rb') as file:
            soup = BeautifulSoup(file)

        for element in soup.find_all('div',attrs={'class':'review review--with-sidebar'}):

            review_dictionary = {} 

            name = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[3]/div[1]/div[1]/h1/text()')
            rating = tree.xpath('//*[@id="wrap"]/div[3]/div/div[1]/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/@title')
            review_dictionary['Restaurant Name'] = ''.join(name).strip()
            review_dictionary['Rating'] = ''.join(rating).strip()

            
            reviewer_raw = element.find('li',attrs={'class':'user-name'})
            text = reviewer_raw.get_text()
            reviewer = text.strip()

            reviewer_num_reviews_raw = element.find('li',attrs={'class':'review-count responsive-small-display-inline-block'})
            text = reviewer_num_reviews_raw.get_text()
            reviewer_num_reviews = text.strip()

            reviewer_elite_status_raw = element.find('li',attrs={'class':'is-elite responsive-small-display-inline-block'})
            if reviewer_elite_status_raw != None:
                text = reviewer_elite_status_raw.get_text()
                reviewer_elite_status = text.strip()

            #extracting reviewer_rating_raw and review_date_raw
            reviewer_rating_raw = element.find('div', attrs={'class':'biz-rating biz-rating-very-large clearfix'}).find('div').find('div')['title']
            reviewer_rating = reviewer_rating_raw.strip()

            #extract the date of the review
            review_date_raw = element.find('div', attrs={'class':'biz-rating biz-rating-very-large clearfix'}).get_text()
            review_date = review_date_raw.strip()

            review_raw = element.find('p')
            text = review_raw.get_text()
            review = text.strip()

            review_dictionary['Reviewer'] = reviewer
            review_dictionary['Reviewers Num of Review'] = reviewer_num_reviews
            review_dictionary['Elite Status'] = reviewer_elite_status
            review_dictionary['Reviewer Rating'] = reviewer_rating
            review_dictionary['Review Date'] = review_date
            review_dictionary['Review'] = review

            #create a variable for the restaurant name to be included in the file name
            restaurant_name =  review_dictionary['Restaurant Name']

            #remove characters that prevent one from creating a unique file name
            restaurant_name = restaurant_name.replace("?", "")

            #save the dictionary for each restaurnat into a csv file 
            save_path = r'C:\Users\Olarn\Desktop\Data Science\Springboard\Yelp_ParsedData\data'
            completeName = os.path.join(save_path, restaurant_name +'_review_num_' + str(restaurant_review_num) +'.csv')
            with open(completeName,'w', encoding = 'utf-8') as handle:
                w = csv.DictWriter(handle,review_dictionary.keys())
                w.writeheader()
                w.writerow(review_dictionary)

            restaurant_review_num += 1

In [ ]:
#converts all csv files saved to hard drive to a pandas data frame for testing
import csv
import glob
import pandas as pd
import chardet

#locate the meta data for restaurants extracted from the html files
path = r'C:\Users\Olarn\Desktop\Data Science\Springboard\Yelp_ParsedData\data'
filenames = glob.glob(os.path.join(path + '\*.csv'))

#extract the encoding information for each csv file 
enc = []
for file in filenames:
    opened_text = open(file,'rb')
    result = chardet.detect(opened_text.read())
    enc.append(result)

#read the file 
dataframe_per_file2 = (pd.read_csv(f, encoding = enc[filenames.index(f)]['encoding']) for f in filenames)

concatenated_df2 = pd.concat(dataframe_per_file2, ignore_index = True)
#concatenated_df2 = concatenated_df[['Restaurant Name', 'Rating', 'Review Date', 'Reviewer Rating', 'Reviewers Num of Reviews', 'Reviewer', 'Elite Status']]
print(concatenated_df2.head())
concatenated_df2.to_csv('yelp_data_v0_1.csv')
